![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Taller: Construcción e implementación de árboles de decisión y métodos de ensamblaje

En este taller podrá poner en práctica los sus conocimientos sobre construcción e implementación de árboles de decisión y métodos de ensamblajes. El taller está constituido por 9 puntos, 5 relacionados con árboles de decisión (parte A) y 4 con métodos de ensamblaje (parte B).

## Parte A - Árboles de decisión

En esta parte del taller se usará el conjunto de datos de Capital Bikeshare de Kaggle, donde cada observación representa el alquiler de bicicletas durante una hora y día determinado. Para más detalles puede visitar los siguientes enlaces: [datos](https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip), [dicccionario de datos](https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset#).

### Datos prestamo de bicicletas

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación de librerías
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, export_graphviz

In [3]:
# Lectura de la información de archivo .csv
bikes = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/bikeshare.csv', index_col='datetime', parse_dates=True)

# Renombrar variable "count" a "total"
bikes.rename(columns={'count':'total'}, inplace=True)

# Crear la hora como una variable 
bikes['hour'] = bikes.index.hour

# Visualización de los datos
bikes.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,total,hour
datetime,,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,0
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,1
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2
2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,3
2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,4


### Punto 1 - Análisis descriptivo

Ejecute las celdas 1.1 y 1.2. A partir de los resultados realice un análisis descriptivo sobre las variables "season" y "hour", escriba sus inferencias sobre los datos. Para complementar su análisis puede usar métricas como máximo, mínimo, percentiles entre otros.

In [ ]:
# Celda 1.1
bikes.groupby('season').total.mean()

Se puede observar que las estaciones 3 (verano) y 2 (primavera) son las que presentan los promedios mas altos en el total de alquiler de bicicleta, seguidos por la estación 4 (otoño) y finalmente la estación 1 (invierno). Esto podría sugerir una relación positiva entre la temporada de clima más cálido y el aumento en el alquiler de bicicletas.

In [ ]:
print(bikes.groupby('season')['total'].describe())

* La estación 3 (verano) tiene una mediana más alta y un rango intercuartílico más amplio en comparación con las otras estaciones, lo que sugiere una distribución más amplia de la demanda durante el verano.

In [ ]:
# Celda 1.2
bikes.groupby('hour').total.mean()

In [ ]:
print(bikes.groupby('hour')['total'].describe())

* Se observa que las horas que presentan mayor demanda en las horas de la mañana son de 7:00 a.m. a 9:00 a.m. y en la tarde de 4:00 p.m. a 7:00 p.m.
* Las horas con la menor demanda de alquiler de bicicletas son durante la noche y las primeras horas de la madrugada, desde la medianoche hasta las 6:00 a.m.

### Punto 2 - Análisis de gráficos

Primero ejecute la celda 2.1 y asegúrese de comprender el código y el resultado. Luego, en cada una de celdas 2.2 y 2.3 escriba un código que genere una gráfica del número de bicicletas rentadas promedio para cada valor de la variable "hour" (hora) cuando la variable "season" es igual a 1 (invierno) e igual a 3 (verano), respectivamente. Analice y escriba sus hallazgos.

In [ ]:
# Celda 2.1 - rentas promedio para cada valor de la variable "hour"
bikes.groupby('hour').total.mean().plot()

In [ ]:
# Celda 2.2 - "season"=1 escriba su código y hallazgos 
bikes_1 = bikes[bikes['season'] == 1]
bikes_1.groupby('hour').total.mean().plot()

Se puede observar que al ser la estación de invierno, los promedios generales de alquiler de bicicletas disminuyen en comparación con el total general, sin hacer distinción por estación. Sin embargo, se evidencian picos de alquiler en las mismas franjas horarias, que van entre las 7:00 a.m. y las 9:00 a.m., así como entre las 4:00 p.m. y las 7:00 p.m.

In [ ]:
# Celda 2.3 - "season"=3 escriba su código y hallazgos 
bikes_3 = bikes[bikes['season'] == 3]
bikes_3.groupby('hour').total.mean().plot()

De manera contraria se observa que para esta estación que corresponde a la de verano, los promedios totales de alquiler de bicicletas están por encima del promedio general. Además, se evidencian picos en las mismas franjas horarias, 
por lo que se puede inferir que independientemente de la estación, se observan picos altos de alquiler en las mismas horas.

### Punto 3 - Regresión lineal
En la celda 3 ajuste un modelo de regresión lineal a todo el conjunto de datos, utilizando "total" como variable de respuesta y "season" y "hour" como las únicas variables predictoras, teniendo en cuenta que la variable "season" es categórica. Luego, imprima los coeficientes e interprételos. ¿Cuáles son las limitaciones de la regresión lineal en este caso?

In [21]:
# Celda 3
bikes_punto3 = pd.get_dummies(bikes, columns=['season'], drop_first=True)

X = bikes_punto3[['hour', 'season_2', 'season_3', 'season_4']]
y = bikes_punto3['total']

model = LinearRegression()
model.fit(X, y)

print("Coeficientes:")
for coef, feature_name in zip(model.coef_, X.columns):
    print(f"{feature_name}: {coef}")

print("\nIntercepto:", model.intercept_)

Coeficientes:
hour: 10.545206094069927
season_2: 100.31723191606622
season_3: 119.46754994593317
season_4: 84.08311787296769

Intercepto: -6.430262462306786


* Con respecto al coeficiente de la hora se puede decir que manteniendo todas las demás variables constantes, por cada aumento de una unidad en la hora del día, se espera un aumento de aproximadamente 10.55 en el número total de bicicletas alquiladas. Esto sugiere que el número de bicicletas alquiladas tiende a aumentar a medida que avanza el día.
* Con respecto a la estación 2 (primavera) se espera que haya aproximadamente 100.32 más bicicletas alquiladas que durante la estación 1 (invierno), manteniendo todas las demás variables constantes.
* Para la estación 3 (verano) se espera que haya aproximadamente 119.46 más bicicletas alquiladas que durante la estación 1 (invierno), manteniendo todas las demás variables constantes.
* Por ultimo para la estación 4 (primavera) se espera que haya aproximadamente 84.08 más bicicletas alquiladas que durante la estación 1 (invierno), manteniendo todas las demás variables constantes.

In [22]:
y_pred = model.predict(X)

# Calcula el R^2
r2 = r2_score(y, y_pred)

# Número de observaciones
n = len(y)

# Número de variables predictoras
p = X.shape[1]

# Calcula el R^2 ajustado
r2_adjusted_punto3 = 1 - (1 - r2) * ((n - 1) / (n - p - 1))

print("R^2 ajustado:", r2_adjusted_punto3)

R^2 ajustado: 0.2231079081517925


**Limitaciones de la regresión lineal**
* La regresión lineal asume que la relación entre las variables predictoras y la variable de respuesta es lineal.por lo que en este caso, la relación entre la hora del día y el número total de bicicletas alquiladas puede no ser necesariamente lineal en todos los casos. Por ejemplo, es posible que la demanda de alquiler de bicicletas aumente de manera no lineal durante las horas pico.
*  La regresión lineal puede ser sensible a valores atípicos en los datos. Si hay valores atípicos presentes en el conjunto de datos, pueden afectar los coeficientes estimados y la precisión del modelo.

### Punto 4 - Árbol de decisión manual
En la celda 4 cree un árbol de decisiones para pronosticar la variable "total" iterando **manualmente** sobre las variables "hour" y  "season". El árbol debe tener al menos 6 nodos finales.

In [9]:
# Celda 4
# Definir parámetros y criterios de parada
X = bikes[['hour', 'season']]
y = bikes['total']

max_depth = None
num_pct = 10
max_features = None
min_gain=0.001

# Definición de la función que calcula el gini index
def gini(y):
    if y.shape[0] == 0:
        return 0
    else:
        return 1 - (y.mean()**2 + (1 - y.mean())**2)



In [10]:
# Función para calcular la ganancia de Gini para una división
def gini_impurity(X_col, y, split):
    
    filter_l = X_col < split
    y_l = y.loc[filter_l]
    y_r = y.loc[~filter_l]
    
    n_l = y_l.shape[0]
    n_r = y_r.shape[0]
    
    gini_y = gini(y)
    gini_l = gini(y_l)
    gini_r = gini(y_r)
    
    gini_impurity_ = gini_y - (n_l / (n_l + n_r) * gini_l + n_r / (n_l + n_r) * gini_r)
    
    return gini_impurity_

In [11]:
# Función para encontrar la mejor división para una variable
def best_split(X, y, num_pct=10):
    
    features = range(X.shape[1])
    
    best_split = [0, 0, 0]  # j, split, gain
    
    # Para todas las varibles 
    for j in features:
        
        splits = np.percentile(X.iloc[:, j], np.arange(0, 100, 100.0 / (num_pct+1)).tolist())
        splits = np.unique(splits)[1:]
        
        # Para cada partición
        for split in splits:
            gain = gini_impurity(X.iloc[:, j], y, split)
                        
            if gain > best_split[2]:
                best_split = [j, split, gain]
    
    return best_split


In [12]:
def tree_grow(X, y, level=0, min_gain=0.001, max_depth=None, num_pct=10):
    
    # Si solo es una observación
    if X.shape[0] == 1:
        tree = dict(y_pred=y.iloc[:1].values[0], y_prob=0.5, level=level, split=-1, n_samples=1, gain=0)
        return tree
    
    # Calcular la mejor división
    j, split, gain = best_split(X, y, num_pct)
    
    # Guardar el árbol y estimar la predicción
    y_pred = int(y.mean() >= 0.5) 
    y_prob = (y.sum() + 1.0) / (y.shape[0] + 2.0)  # Corrección Laplace 
    
    tree = dict(y_pred=y_pred, y_prob=y_prob, level=level, split=-1, n_samples=X.shape[0], gain=gain)
    # Revisar el criterio de parada 
    if gain < min_gain:
        return tree
    if max_depth is not None:
        if level >= max_depth:
            return tree   
    
    # Continuar creando la partición
    filter_l = X.iloc[:, j] < split
    X_l, y_l = X.loc[filter_l], y.loc[filter_l]
    X_r, y_r = X.loc[~filter_l], y.loc[~filter_l]
    tree['split'] = [j, split]

    # Siguiente iteración para cada partición
    
    tree['sl'] = tree_grow(X_l, y_l, level + 1, min_gain=min_gain, max_depth=max_depth, num_pct=num_pct)
    tree['sr'] = tree_grow(X_r, y_r, level + 1, min_gain=min_gain, max_depth=max_depth, num_pct=num_pct)
    
    return tree

In [13]:
# Construir el árbol de decisiones
tree_grow(X, y, level=0, min_gain=0.001, max_depth=6, num_pct=10)


{'y_pred': 1,
 'y_prob': 191.53903379867745,
 'level': 0,
 'split': [0, 8.0],
 'n_samples': 10886,
 'gain': 18268.811823533004,
 'sl': {'y_pred': 1,
  'y_prob': 55.40711902113459,
  'level': 1,
  'split': [0, 7.0],
  'n_samples': 3594,
  'gain': 7207.700659959655,
  'sl': {'y_pred': 1,
   'y_prob': 32.561604584527224,
   'level': 2,
   'split': [0, 6.0],
   'n_samples': 3139,
   'gain': 646.8008927589567,
   'sl': {'y_pred': 1,
    'y_prob': 25.15934475055845,
    'level': 3,
    'split': [0, 2.0],
    'n_samples': 2684,
    'gain': 382.8088308604629,
    'sl': {'y_pred': 1,
     'y_prob': 44.41383095499451,
     'level': 4,
     'split': [1, 2.0],
     'n_samples': 909,
     'gain': 291.4004681825131,
     'sl': {'y_pred': 1,
      'y_prob': 23.32456140350877,
      'level': 5,
      'split': [0, 1.0],
      'n_samples': 226,
      'gain': 45.41972746495435,
      'sl': {'y_pred': 1,
       'y_prob': 27.808695652173913,
       'level': 6,
       'split': -1,
       'n_samples': 113,
 

### Punto 5 - Árbol de decisión con librería
En la celda 5 entrene un árbol de decisiones con la **librería sklearn**, usando las variables predictoras "season" y "hour" y calibre los parámetros que considere conveniente para obtener un mejor desempeño. Recuerde dividir los datos en conjuntos de entrenamiento y validación para esto. Comente el desempeño del modelo con alguna métrica de desempeño de modelos de regresión y compare desempeño con el modelo del punto 3.

In [25]:
# Celda 5
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

tree_model = DecisionTreeRegressor(random_state=42)

# Entrenar el modelo
tree_model.fit(X_train, y_train)

# Predecir en el conjunto de validación
y_pred = tree_model.predict(X_test)

# Entrenar el modelo
tree_model.fit(X_train, y_train)

# Predecir en el conjunto de validación
y_pred = tree_model.predict(X_test)

r2 = r2_score(y_test, y_pred)

# Número de observaciones
n = len(y_test)

# Número de variables predictoras
p = X_test.shape[1]

# Calcula el R^2 ajustado
r2_adjusted_punto5 = 1 - (1 - r2) * ((n - 1) / (n - p - 1))

print("R^2 ajustado árbol de decisión:", r2_adjusted_punto5)
print("R^2 ajustado Regresión lienal:", r2_adjusted_punto3)

R^2 ajustado árbol de decisión: 0.5761010614049551
R^2 ajustado Regresión lienal: 0.2231079081517925


El árbol de decisión tiene un R^2 ajustado de aproximadamente 0.576, lo que significa que alrededor del 57.6% de la variabilidad en la variable de respuesta es explicada por el modelo. Mientras que la regresión lineal tiene un R^2 ajustado de aproximadamente 0.223, lo que indica que solo alrededor del 22.3% de la variabilidad es explicada por el modelo lineal. por lo que basándose en los valores de R^2 ajustado proporcionados, se puede concluir que el modelo de árbol de decisión tiene un mejor rendimiento en este conjunto de datos en comparación con el modelo de regresión lineal.

## Parte B - Métodos de ensamblajes
En esta parte del taller se usará el conjunto de datos de Popularidad de Noticias Online. El objetivo es predecir si la notica es popular o no, la popularidad está dada por la cantidad de reacciones en redes sociales. Para más detalles puede visitar el siguiente enlace: [datos](https://archive.ics.uci.edu/ml/datasets/online+news+popularity).

### Datos popularidad de noticias

In [26]:
# Lectura de la información de archivo .csv
df = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/mashable.csv', index_col=0)
df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2014/12/10/cia-torture-rep...,28.0,9.0,188.0,0.732620,1.0,0.844262,5.0,1.0,1.0,...,0.200000,0.80,-0.487500,-0.60,-0.250000,0.9,0.8,0.4,0.8,1
1,http://mashable.com/2013/10/18/bitlock-kicksta...,447.0,7.0,297.0,0.653199,1.0,0.815789,9.0,4.0,1.0,...,0.160000,0.50,-0.135340,-0.40,-0.050000,0.1,-0.1,0.4,0.1,0
2,http://mashable.com/2013/07/24/google-glass-po...,533.0,11.0,181.0,0.660377,1.0,0.775701,4.0,3.0,1.0,...,0.136364,1.00,0.000000,0.00,0.000000,0.3,1.0,0.2,1.0,0
3,http://mashable.com/2013/11/21/these-are-the-m...,413.0,12.0,781.0,0.497409,1.0,0.677350,10.0,3.0,1.0,...,0.100000,1.00,-0.195701,-0.40,-0.071429,0.0,0.0,0.5,0.0,0
4,http://mashable.com/2014/02/11/parking-ticket-...,331.0,8.0,177.0,0.685714,1.0,0.830357,3.0,2.0,1.0,...,0.100000,0.55,-0.175000,-0.25,-0.100000,0.0,0.0,0.5,0.0,0


In [27]:
# Definición variable de interes y variables predictoras
X = df.drop(['url', 'Popular'], axis=1)
y = df['Popular']
y.mean()

0.5

In [28]:
# División de la muestra en set de entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Punto 6 - Árbol de decisión y regresión logística
En la celda 6 construya un árbol de decisión y una regresión logística. Para el árbol calibre al menos un parámetro y evalúe el desempeño de cada modelo usando las métricas de Accuracy y F1-Score.

In [37]:
# Celda 6

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth': range(1, 20)}  

# Inicializar el clasificador de árbol de decisión
tree_model = DecisionTreeClassifier(random_state=1)

grid_search = GridSearchCV(tree_model, param_grid, cv=5, scoring='f1')

# Entrenar GridSearchCV
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo después de la calibración
best_tree_model = grid_search.best_estimator_

# Predecir utilizando el mejor modelo
y_pred_tree = best_tree_model.predict(X_test)

# Calcular métricas de evaluación
accuracy_tree = accuracy_score(y_test, y_pred_tree)
f1_tree = f1_score(y_test, y_pred_tree)


# # Regresion logistica
log_reg_model = LogisticRegression()

# # Entrenamiento regresion logistica
log_reg_model.fit(X_train, y_train)
y_pred_RL = log_reg_model.predict(X_test)
accuracy_RL = accuracy_score(y_test, y_pred_RL)
f1_RL = f1_score(y_test, y_pred_RL)

print("Mejor parámetro para el árbol de decisión:", grid_search.best_params_)
print("Accuracy del árbol:", accuracy_tree)
print("F1-score del árbol:", f1_tree)
print("Accuracy de regresión logistica:", accuracy_RL)
print("F1-score de regresión logistica:", f1_RL)



Mejor parámetro para el árbol de decisión: {'max_depth': 4}
Accuracy del árbol: 0.6513333333333333
F1-score del árbol: 0.6345213137665969
Accuracy de regresión logistica: 0.614
F1-score de regresión logistica: 0.6106254203093476


* Se observa que el árbol de decisión tiene una mejor precisión general en la clasificación de las muestras en comparación con el modelo de regresión logística.
* El mel árbol de decisión parece tener un mejor desempeño que el modelo de regresión logística en este conjunto de datos específico, teniendo en cuenta ambas métricas de evaluación.

### Punto 7 - Votación Mayoritaria
En la celda 7 elabore un esamble con la metodología de **Votación mayoritaria** compuesto por 300 muestras bagged donde:

-las primeras 100 muestras vienen de árboles de decisión donde max_depth tome un valor de su elección\
-las segundas 100 muestras vienen de árboles de decisión donde min_samples_leaf tome un valor de su elección\
-las últimas 100 muestras vienen de regresiones logísticas

Evalúe cada uno de los tres modelos de manera independiente utilizando las métricas de Accuracy y F1-Score, luego evalúe el ensamble de modelos y compare los resultados. 

Nota: 

Para este ensamble de 300 modelos, deben hacer votación mayoritaria. Esto lo pueden hacer de distintas maneras. La más "fácil" es haciendo la votación "manualmente", como se hace a partir del minuto 5:45 del video de Ejemplo práctico de emsablajes en Coursera. Digo que es la más fácil porque si hacen la votación mayoritaria sobre las 300 predicciones van a obtener lo que se espera.

Otra opción es: para cada uno de los 3 tipos de modelos, entrenar un ensamble de 100 modelos cada uno. Predecir para cada uno de esos tres ensambles y luego predecir como un ensamble de los 3 ensambles. La cuestión es que la votación mayoritaria al usar los 3 ensambles no necesariamente va a generar el mismo resultado que si hacen la votación mayoritaria directamente sobre los 300 modelos. Entonces, para los que quieran hacer esto, deben hacer ese último cálculo con cuidado.

Para los que quieran hacerlo como ensamble de ensambles, digo que se debe hacer el ensamble final con cuidado por lo siguiente. Supongamos que:

* para los 100 árboles del primer tipo, la votación mayoritaria es: 55% de los modelos predicen que la clase de una observación es "1"
* para los 100 árboles del segundo tipo, la votación mayoritaria es: 55% de los modelos predicen que la clase de una observación es "1"
* para las 100 regresiones logísticas, la votación mayoritaria es: 10% de los modelos predicen que la clase de una observación es "1"

Si se hace la votación mayoritaria de los 300 modelos, la predicción de esa observación debería ser: (100*55%+100*55%+100*10%)/300 = 40% de los modelos votan porque la predicción debería ser "1". Es decir, la predicción del ensamble es "0" (dado que menos del 50% de modelos predijo un 1).

Sin embargo, si miramos cada ensamble por separado, el primer ensamble predice "1", el segundo ensamble predice "1" y el último ensamble predice "0". Si hago votación mayoritaria sobre esto, la predicción va a ser "1", lo cual es distinto a si se hace la votación mayoritaria sobre los 300 modelos.

In [42]:
# Celda 7
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# árboles de decisión con max_depth específico
tree_models_max_depth = []
for _ in range(100):
    tree_model_max_depth = DecisionTreeClassifier(max_depth=4) 
    tree_model_max_depth.fit(X_train, y_train)
    tree_models_max_depth.append(tree_model_max_depth)

#  árboles de decisión con min_samples_leaf específico
tree_models_min_samples_leaf = []
for _ in range(100):
    tree_model_min_samples_leaf = DecisionTreeClassifier(min_samples_leaf=20)  
    tree_model_min_samples_leaf.fit(X_train, y_train)
    tree_models_min_samples_leaf.append(tree_model_min_samples_leaf)

# regresión logística
log_reg_models = []
for _ in range(100):
    log_reg_model = LogisticRegression()
    log_reg_model.fit(X_train, y_train)
    log_reg_models.append(log_reg_model)

# Generar predicciones para cada subconjunto de modelos
predictions_max_depth = [model.predict(X_test) for model in tree_models_max_depth]
predictions_min_samples_leaf = [model.predict(X_test) for model in tree_models_min_samples_leaf]
predictions_log_reg = [model.predict(X_test) for model in log_reg_models]

# Realizar la votación mayoritaria para cada subconjunto de modelos
ensemble_predictions_max_depth = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=predictions_max_depth)
ensemble_predictions_min_samples_leaf = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=predictions_min_samples_leaf)
ensemble_predictions_log_reg = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=predictions_log_reg)

# Evaluar cada subconjunto de modelos
accuracy_tree_max_depth = accuracy_score(y_test, ensemble_predictions_max_depth)
f1_tree_max_depth = f1_score(y_test, ensemble_predictions_max_depth)
accuracy_tree_min_samples_leaf = accuracy_score(y_test, ensemble_predictions_min_samples_leaf)
f1_tree_min_samples_leaf = f1_score(y_test, ensemble_predictions_min_samples_leaf)
accuracy_log_reg = accuracy_score(y_test, ensemble_predictions_log_reg)
f1_log_reg = f1_score(y_test, ensemble_predictions_log_reg)

# Realizar la votación mayoritaria para el ensamble final
final_predictions = np.column_stack((ensemble_predictions_max_depth, ensemble_predictions_min_samples_leaf, ensemble_predictions_log_reg))
ensemble_predictions = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=1, arr=final_predictions)

######3 Evaluar el ensamble final
accuracy_ensemble = accuracy_score(y_test, ensemble_predictions)
f1_ensemble = f1_score(y_test, ensemble_predictions)

# métricas de evaluación
print("Métricas de evaluación para el subconjunto de árboles de decisión con max_depth específico:")
print("Accuracy:", accuracy_tree_max_depth)
print("F1-score:", f1_tree_max_depth)
print("\nMétricas de evaluación para el subconjunto de árboles de decisión con min_samples_leaf específico:")
print("Accuracy:", accuracy_tree_min_samples_leaf)
print("F1-score:", f1_tree_min_samples_leaf)
print("\nMétricas de evaluación para el subconjunto de regresión logística:")
print("Accuracy:", accuracy_log_reg)
print("F1-score:", f1_log_reg)
print("\nMétricas de evaluación para el ensamble final:")
print("Accuracy del ensamble:", accuracy_ensemble)
print("F1-score del ensamble:", f1_ensemble)


Métricas de evaluación para el subconjunto de árboles de decisión con max_depth específico:
Accuracy: 0.6513333333333333
F1-score: 0.6345213137665969

Métricas de evaluación para el subconjunto de árboles de decisión con min_samples_leaf específico:
Accuracy: 0.57
F1-score: 0.551772063933287

Métricas de evaluación para el subconjunto de regresión logística:
Accuracy: 0.614
F1-score: 0.6106254203093476

Métricas de evaluación para el ensamble final:
Accuracy del ensamble: 0.642
F1-score del ensamble: 0.629399585921325


* Se observa que el subconjunto de árboles de decisión con max_depth específico tiene el mejor desempeño en términos de precisión (Accuracy) y F1-score en comparación con los otros dos subconjuntos de modelos.
* El subconjunto de árboles de decisión con min_samples_leaf específico tiene el peor desempeño, con una precisión y F1-score más bajos.

### Punto 8 - Votación Ponderada
En la celda 8 elabore un ensamble con la metodología de **Votación ponderada** compuesto por 300 muestras bagged para los mismos tres escenarios del punto 7. Evalúe los modelos utilizando las métricas de Accuracy y F1-Score

In [ ]:
# Celda 8


### Punto 9 - Comparación y análisis de resultados
En la celda 9 comente sobre los resultados obtenidos con las metodologías usadas en los puntos 7 y 8, compare los resultados y enuncie posibles ventajas o desventajas de cada una de ellas.

In [ ]:
# Celda 9